# Example 2: Basic usage of treeview

Simple example of a user jupyter notebook that fills and displays three histograms from simulation, saved as a ROOT tree chain.

In [1]:
import ROOT
%jsroot on
#%pip install --user "gluex.jupyroot>=1.0.4"
from gluex.jupyroot.treeview import treeview

In [2]:
xrdurl = "root://cn445.storrs.hpc.uconn.edu"
xrdpath = "/Gluex/resilient/simulation/KLFbeam-8-2024/"
infile = "bgprofiles5_{0}.root"
rfile1 = ROOT.TFile.Open(xrdurl + xrdpath + infile.format(1))
tree = rfile1.Get("det5")
chain = ROOT.TChain(tree.GetName(), tree.GetTitle())
[chain.Add(xrdurl + xrdpath + infile.format(i+1)) for i in range(100)]
tview = treeview(chain, "examples.root/example2") # to clear the cache, remove this root file and rerun the notebook

In [3]:
fill_histograms_in_parallel = False
if fill_histograms_in_parallel:
    import dask
    import dask.distributed
    dclient = dask.distributed.Client(n_workers=5, threads_per_worker=1, dashboard_address='0.0.0.0:8787')
    tview.enable_dask_cluster(dclient)

/srv/.local/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41653 instead
  warnings.warn(


In [4]:
def run_stats_hinit():
    h = {}
    h['hrunno'] = ROOT.TH1D('hrunno', "run number, all events", 10, 120000, 120010)
    h['hrunno'].GetXaxis().SetTitle("run")
    h['hrunno'].GetYaxis().SetTitle("count")
    h['hevtno'] = ROOT.TH1D('hevtno', "event number, all events", 500, 0, 500000)
    h['hevtno'].GetXaxis().SetTitle("event")
    h['hevtno'].GetYaxis().SetTitle("count")
    h['hptype'] = ROOT.TH1D('hptype', "primary particle types", 100, 0, 100)
    h['hptype'].GetXaxis().SetTitle("G3 particle code")
    h['hptype'].GetYaxis().SetTitle("count")
    return h

def run_stats_hfill(row, histos):
    """
    To list the contents of the tree columns, just do print(row),
    here we mine the contents of the record for relevant information.
    """
    histos['hrunno'].Fill(row.run)
    histos['hevtno'].Fill(row.event)
    histos['hptype'].Fill(99)
    return histos

tview.declare_histograms("run statistics", run_stats_hinit, run_stats_hfill)
n = tview.fill_histograms()
n = tview.draw([['hrunno', 'hevtno'], ['hptype', ]], width=600)            

found 4 histograms that need filling, follow progress on dask monitor dashboard at http://cn447.storrs.hpc.uconn.edu:41653/status
fill_histograms read a total of 100 tree files, 294552.0 records
  OBJ: TH1D	hrunno	run number, all events : 0 at: 0x557ec15caaa0
 OBJ: TH1D	hevtno	event number, all events : 0 at: 0x557ec17e4670
 OBJ: TH1D	hptype	primary particle types : 0 at: 0x557ec221f810
